# 2. Ансамбли моделей. Бутстреппинг. Бэггинг

 Задание 2.7
0.0/2.0 points (graded)

Объёмная и содержательная практика у нас ещё впереди, но в качестве разминки давайте поработаем с уже известным вам датасетом о вине, который можно скачать здесь.

In [2]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные модели
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики

from sklearn.model_selection import train_test_split

In [3]:
wine_data = pd.read_csv('data/wineQualityReds.csv')
wine_data.head()

,Unnamed: 0,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
X = wine_data.drop(['Unnamed: 0','quality'], axis=1)
y = wine_data['quality'].apply(lambda x: 1 if x>=6 else 0)

In [5]:
y.value_counts()

quality
1    855
0    744
Name: count, dtype: int64

In [6]:
X.isna().sum().sum()

0

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

логистическую регрессию (с параметрами по умолчанию) 

In [8]:
lr = linear_model.LogisticRegression()
lr.fit(X_train, y_train)
#
y_pred = lr.predict(X_test)
print('F1_score', round(metrics.f1_score(y_pred, y_test),3))

F1_score 0.754


d:\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


дерево решений (random_state = 42, максимальная глубина — 10).

In [9]:
dt = tree.DecisionTreeClassifier(random_state=42, max_depth=10)
dt.fit(X_train, y_train)
#
y_pred_dt = dt.predict(X_test)
print('F1_score', round(metrics.f1_score(y_pred_dt, y_test),3))

F1_score 0.793


 Задание 2.8
1 point possible (graded)

Обучите модель с использованием бэггинга (класс BaggingClassifier с random_state=42).

Возьмите из предыдущего задания алгоритм, показавший наилучшее качество, и укажите для него новое количество моделей — 1500. Вычислите новое значение F1-score.

In [10]:
bc_dt = ensemble.BaggingClassifier(
    estimator=dt,
    n_estimators=1500,
    random_state=42)

bc_dt.fit(X_train, y_train)
#
y_pred_bc_dt = bc_dt.predict(X_test)
print('F1_score', round(metrics.f1_score(y_pred_bc_dt, y_test),3))

F1_score 0.818


# 3. Случайный лес

In [11]:
houses_data = pd.read_csv('data/boston (1).csv')
houses_data.head()

,crim_rate,zn,business,river,nit_oxiden,rooms,age,dist,highways_index,tax,pup_per_teaс,lower,target
0,"0,00632",18,"2,31",0,"0,538","6,575","65,2","4,09",1,296,"15,3","4,98",24
1,"0,02731",0,"7,07",0,"0,469","6,421","78,9","4,9671",2,242,"17,8","9,14","21,6"
2,"0,02729",0,"7,07",0,"0,469","7,185","61,1","4,9671",2,242,"17,8","4,03","34,7"
3,"0,03237",0,"2,18",0,"0,458","6,998","45,8","6,0622",3,222,"18,7","2,94","33,4"
4,"0,06905",0,"2,18",0,"0,458","7,147","54,2","6,0622",3,222,"18,7","5,33","36,2"


In [12]:
houses_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   crim_rate       506 non-null    object
 1   zn              506 non-null    object
 2   business        506 non-null    object
 3   river           506 non-null    int64 
 4   nit_oxiden      506 non-null    object
 5   rooms           506 non-null    object
 6   age             506 non-null    object
 7   dist            506 non-null    object
 8   highways_index  506 non-null    int64 
 9   tax             506 non-null    int64 
 10  pup_per_teaс    506 non-null    object
 11  lower           506 non-null    object
 12  target          506 non-null    object
dtypes: int64(3), object(10)
memory usage: 51.5+ KB


In [13]:
def conv(name, field):
    name[field] = name[field].str.replace(',','.')
    name[field] = name[field].astype('Float64')
#
def conv_int(name, field):
    name[field] = name[field].str.replace(',','.')
    name[field] = name[field].astype('int64')    

In [14]:
conv(houses_data, 'crim_rate')
conv(houses_data, 'business')
conv(houses_data, 'nit_oxiden')
conv(houses_data, 'rooms')
conv(houses_data, 'age')
conv(houses_data, 'dist')
conv(houses_data, 'pup_per_teaс')
conv(houses_data, 'lower')
conv(houses_data, 'target')
conv(houses_data, 'zn')

In [15]:
houses_data['target']

0      24.0
1      21.6
2      34.7
3      33.4
4      36.2
       ... 
501    22.4
502    20.6
503    23.9
504    22.0
505    11.9
Name: target, Length: 506, dtype: Float64

In [16]:
houses_data.isna().sum()

crim_rate         0
zn                0
business          0
river             0
nit_oxiden        0
rooms             0
age               0
dist              0
highways_index    0
tax               0
pup_per_teaс      0
lower             0
target            0
dtype: int64

Разбейте набор данных на обучающую и тестовую выборку в соотношении 70/30, при разбиении задайте параметр random_state = 13.

In [17]:
X = houses_data.drop(['target'], axis=1)
y = houses_data['target']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13, test_size=0.3)

In [19]:
round(y_train.mean(),2)

22.77

2. Обучите линейную регрессию с параметрами по умолчанию.

In [20]:
lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)
#
y_pred = lr.predict(X_test)
print('MAE_score', round(metrics.mean_absolute_error(y_pred, y_test),2))

MAE_score 3.72


3. Обучите решающее дерево с параметрами по умолчанию и аргументом random_state = 13.

Можно ли, опираясь на результаты, сделать вывод, что алгоритм переобучился?

In [21]:
dt = tree.DecisionTreeRegressor(random_state=13)
dt.fit(X_train, y_train)
#
y_pred_dt = dt.predict(X_test)
print('MAE_score', round(metrics.mean_absolute_error(y_pred, y_test),2))

MAE_score 3.72


4. Обучите четыре случайных леса с числом деревьев 3, 10, 100, 500 и параметром random_state = 13.

In [22]:
for i in [3,10,100,500]:
    rf = ensemble.RandomForestRegressor(random_state=13, n_estimators=i)
    rf.fit(X_train, y_train)
    #
    y_pred_rf = rf.predict(X_test)
    print('trees=', i, 'MAE_score=', round(metrics.mean_absolute_error(y_pred_rf, y_test),2))

trees= 3 MAE_score= 2.93
trees= 10 MAE_score= 2.47
trees= 100 MAE_score= 2.26
trees= 500 MAE_score= 2.24


# 4. Случайный лес. Практика

In [23]:
data = pd.read_csv('data/weatherAUS.csv')
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [24]:
data.isna().sum().sum()

343248

In [25]:
data.isna().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [26]:
n = data.shape[0] * 0.4

In [27]:
n = data.shape[0] * 0.6
data_drop = data.dropna(axis=1, thresh=n)

In [28]:
data.isna().sum() / len(data)

Date             0.000000
Location         0.000000
MinTemp          0.010209
MaxTemp          0.008669
Rainfall         0.022419
Evaporation      0.431665
Sunshine         0.480098
WindGustDir      0.070989
WindGustSpeed    0.070555
WindDir9am       0.072639
WindDir3pm       0.029066
WindSpeed9am     0.012148
WindSpeed3pm     0.021050
Humidity9am      0.018246
Humidity3pm      0.030984
Pressure9am      0.103568
Pressure3pm      0.103314
Cloud9am         0.384216
Cloud3pm         0.408071
Temp9am          0.012148
Temp3pm          0.024811
RainToday        0.022419
RainTomorrow     0.022460
dtype: float64

In [29]:
data.isna().sum().count() - data_drop.isna().sum().count()

3

 Задание 4.3
1 point possible (graded)

Теперь обработаем признаки RainToday и RainTomorrow таким образом, чтобы вместо yes было значение 1, а вместо no — значение 0. Обратите внимание на то, что в признаке RainTomorrow присутствуют пропуски, и их трогать не нужно, они должны остаться пропусками. Поэтому обрабатывайте столбцы таким образом, чтобы не видоизменить пропущенные значения.

Вычислите среднее арифметическое для преобразованного признака RainToday и запишите его в ответ, предварительно округлив до двух знаков после точки-разделителя.

In [30]:
data.RainToday = data.RainToday.map({'No': 0, 'Yes': 1})
data.RainTomorrow = data.RainTomorrow.map({'No': 0, 'Yes': 1})

In [31]:
data['RainTomorrow'].value_counts()

RainTomorrow
0.0    110316
1.0     31877
Name: count, dtype: int64

In [32]:
data_drop['RainTomorrow'].isna().sum()

3267

In [33]:
round(data['RainToday'].mean(),2)

0.22

 Задание 4.4
1 point possible (graded)
Обработайте признак Date таким образом, чтобы выделить в отдельный признак Month (номер месяца). Изначальный признак Date удалите. Определите, в какой месяц в среднем за день выпадает больше всего дождей. В качестве ответа введите порядковый номер месяца.

In [34]:
data['Date'] = pd.to_datetime(data['Date'])
data['Month'] = data['Date'].dt.month

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           145460 non-null  datetime64[ns]
 1   Location       145460 non-null  object        
 2   MinTemp        143975 non-null  float64       
 3   MaxTemp        144199 non-null  float64       
 4   Rainfall       142199 non-null  float64       
 5   Evaporation    82670 non-null   float64       
 6   Sunshine       75625 non-null   float64       
 7   WindGustDir    135134 non-null  object        
 8   WindGustSpeed  135197 non-null  float64       
 9   WindDir9am     134894 non-null  object        
 10  WindDir3pm     141232 non-null  object        
 11  WindSpeed9am   143693 non-null  float64       
 12  WindSpeed3pm   142398 non-null  float64       
 13  Humidity9am    142806 non-null  float64       
 14  Humidity3pm    140953 non-null  float64       
 15  

In [36]:
data.drop('Date', axis = 1, inplace = True)
df_season = data.groupby('Month')['RainToday'].mean()
df_season

,RainToday
Month,
1,0.189484
2,0.206746
3,0.217135
4,0.216845
5,0.222163
6,0.263638
7,0.270736
8,0.253167
9,0.229135


 Задание 4.5
1 point possible (graded)

Обработайте оставшиеся категориальные признаки. С помощью метода get_dummies с настройками по умолчанию создайте dummy-переменные для всех категориальных признаков (их пять), которые есть в данных на этот момент.

Кодировку признаков важно выполнить именно в следующем порядке: categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']. Это необходимо для того, чтобы ваши дальнейшие ответы сходились с нашим решением, так как алгоритм случайного леса, который мы будем использовать в дальнейшем, чувствителен к порядку столбцов.

Сколько теперь признаков в данных, если считать целевую переменную?

In [38]:
categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
df_dummies = pd.get_dummies(data, columns=categoricals)
df_dummies.shape

(145460, 127)

 Задание 4.6
1 point possible (graded)

Осталось совсем немного. Удалите все строки, где есть пропуски. Далее разбейте данные на обучающую и тестовую выборки в соотношении 70/30, в качестве значения параметра random_state возьмите число 31.

Каково среднее значение целевой переменной на тестовой выборке? Ответ округлите до двух знаков после точки-разделителя.

In [43]:
df_dummies.dropna(inplace=True)
X = df_dummies.drop('RainTomorrow', axis = 1)
Y = df_dummies['RainTomorrow']  
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 31)
Y_test.mean()

0.21845412291272165